In [3]:
import numpy as np
import mediapipe as mp
import cv2

In [4]:
# Initialize MediaPipe
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

In [5]:
def calculate_distance(p1, p2):
    return np.linalg.norm(np.array(p1) - np.array(p2))

In [6]:
# Initialize video capture (0 = default camera)
cap = cv2.VideoCapture(0)

# Check if camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Loop to capture frames
while True:
    ret, frame = cap.read()  # Read a frame
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Display the frame
    cv2.imshow('Live Video', frame)

    # Exit on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

In [7]:
# Initialize video capture (0 = default camera)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert image to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    h, w, _ = frame.shape

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Get Lip Landmarks
            upper_lip = (int(face_landmarks.landmark[11].x * w), int(face_landmarks.landmark[11].y * h))
            lower_lip = (int(face_landmarks.landmark[16].x * w), int(face_landmarks.landmark[16].y * h))
            
            # Calculate lip distance
            lip_distance = calculate_distance(upper_lip, lower_lip)

            # cv2.putText(frame, f"{lip_distance}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Detect lip biting (threshold ~3 pixels)
            if lip_distance < 10:
                cv2.putText(frame, "Lip Biting Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Get Eye Center Landmarks
            left_eye = (int(face_landmarks.landmark[468].x * w), int(face_landmarks.landmark[468].y * h))
            right_eye = (int(face_landmarks.landmark[473].x * w), int(face_landmarks.landmark[473].y * h))

            # Draw landmarks
            cv2.circle(frame, upper_lip, 3, (0, 255, 0), -1)
            cv2.circle(frame, lower_lip, 3, (0, 255, 0), -1)
            cv2.circle(frame, left_eye, 3, (255, 0, 0), -1)
            cv2.circle(frame, right_eye, 3, (255, 0, 0), -1)

    # Show Frame
    cv2.imshow("Lip & Eye Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [8]:
from collections import deque

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

# Store past eye positions to track movement
eye_positions = deque(maxlen=5)  # Stores last 5 eye positions

def calculate_distance(p1, p2):
    """Calculate Euclidean distance between two points"""
    return np.linalg.norm(np.array(p1) - np.array(p2))

# Open Webcam
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    h, w, _ = frame.shape  # Get frame dimensions

    # Convert frame to RGB for MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Get Eye Center Landmarks
            left_eye = (int(face_landmarks.landmark[468].x * w), int(face_landmarks.landmark[468].y * h))
            right_eye = (int(face_landmarks.landmark[473].x * w), int(face_landmarks.landmark[473].y * h))

            # Calculate eye center (average of both eyes)
            eye_center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)

            # Append new eye position to queue
            eye_positions.append(eye_center)

            # Check movement if we have enough frames
            if len(eye_positions) == 5:
                total_movement = sum(
                    calculate_distance(eye_positions[i], eye_positions[i + 1]) for i in range(len(eye_positions) - 1)
                )

                # If movement is too high, classify as attention loss
                if total_movement > 10:  # Adjust threshold as needed
                    cv2.putText(frame, " Attention Loss Detected!", (50, 100),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Draw landmarks for visualization (Optional)
            cv2.circle(frame, left_eye, 3, (255, 0, 0), -1)
            cv2.circle(frame, right_eye, 3, (255, 0, 0), -1)

    # Display Frame
    cv2.imshow("Eye Movement Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [9]:
import time

In [16]:
def rel_position_eye(eye_landmarks,iris_landmarks):
    # gives value [0,1] independent of frame width, can check eye motion
    eye_left = eye_landmarks[0]#left corner of eye
    eye_right = eye_landmarks[1]#right corner of eye
    eye_iris=iris_landmarks


    eye_range = np.linalg.norm(np.array(eye_left) - np.array(eye_right))
    iris_left_dist=np.linalg.norm(np.array(eye_left) - np.array(eye_iris))
    return iris_left_dist/eye_range

gaze_history = deque(maxlen=30)
# Initialize video capture (0 = default camera)
cap = cv2.VideoCapture(0)
start = time.time()
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert image to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    h, w, _ = frame.shape

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            

            left_eye = [(int(face_landmarks.landmark[33].x * w), int(face_landmarks.landmark[33].y * h)),  # Left corner
                        (int(face_landmarks.landmark[133].x * w), int(face_landmarks.landmark[133].y * h))]  # Right corner
            right_eye = [(int(face_landmarks.landmark[362].x * w), int(face_landmarks.landmark[362].y * h)),  
                         (int(face_landmarks.landmark[263].x * w), int(face_landmarks.landmark[263].y * h))]
            right_iris=[(int(face_landmarks.landmark[473].x * w), int(face_landmarks.landmark[473].y * h))]
            left_iris=[(int(face_landmarks.landmark[468].x * w), int(face_landmarks.landmark[468].y * h))]

            left_iris_pos= rel_position_eye(left_eye,left_iris)
            right_iris_pos= rel_position_eye(right_eye,right_iris)


            # Determine gaze direction
            gaze_direction = "Center"  # Default
            if left_iris_pos < 0.4 and right_iris_pos < 0.4:
                gaze_direction = "Looking Right"
            elif left_iris_pos > 0.60 and right_iris_pos > 0.60:
                gaze_direction = "Looking Left"
            elif face_landmarks.landmark[468].y  > face_landmarks.landmark[33].y *(1+0.02):  
                gaze_direction = "Looking Down"

            # Store last few gaze directions
            gaze_history.append(gaze_direction)

            # Check if user is looking away for too long
            if gaze_history.count("Looking Right") > 11 or gaze_history.count("Looking Left") > 11:
                cv2.putText(frame, "Distraction Detected!", (50, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif gaze_history.count("Looking Down") > 11:
                cv2.putText(frame, "Possible Sleepiness!", (50, 150),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

            # Draw landmarks

            for (x, y) in left_eye + right_eye + left_iris + right_iris:
                cv2.circle(frame, (int(x), int(y)), 3, (0, 255, 0), -1)

            # Display gaze direction
            cv2.putText(frame, f"Gaze: {gaze_direction}", (50, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)


            # cv2.putText(frame, f"{rel_position_eye(left_eye,left_iris)}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # print(f"left = {rel_position_eye(left_eye,left_iris)}","\t",f"right = {rel_position_eye(right_eye,right_iris)}")

    # Show Frame
    cv2.imshow("Eye Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
end=time.time()
print(end-start,h,w,_)
cap.release()
cv2.destroyAllWindows()

17.15454411506653 480 640 3


In [12]:
gaze_history

deque(['Center',
       'Center',
       'Center',
       'Center',
       'Center',
       'Center',
       'Center',
       'Center',
       'Center',
       'Center'],
      maxlen=10)